In [45]:
import sqlite3
import pandas as pd
import altair as alt
import numpy as np
from sklearn.cluster import DBSCAN

In [16]:
conn = sqlite3.connect('G:\\내 드라이브\\sync DeskTop-LapTop (2024.08)\\files\\data\\상권분석\\상권분석.db')

In [12]:
conn

In [17]:
c = conn.cursor()

In [15]:
c.execute('''
          SELECT name FROM sqlite_master WHERE type='table';
          ''')

# for row in c.fetchall():
#     print(row)
c.fetchall()

[]

In [18]:
query = 'select *  from Market_info limit 1'

column_check = pd.read_sql_query(query, conn)
column_check

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA0101202210A0084547,금강산노래광장,None,I2,음식,I211,주점,I21101,일반 유흥 주점,I56211,...,4.217010e+24,파크장,강원특별자치도 동해시 송정로 11,240806,25789,None,지,None,129.127525,37.495265


In [22]:
column_check.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [19]:
## 각 지역의 인구수와 상가 수를 비교합니다.
query = '''
    SELECT 상권업종대분류명
         , 시도명
         , 시군구명
         , 행정동명
         , 법정동명
         , count(*) as cnt
         from Market_info
         GROUP BY 상권업종대분류명
         , 시도명
         , 시군구명
         , 행정동명
         , 법정동명
'''

df_market_cnt_by_region = pd.read_sql_query(query, conn)
df_market_cnt_by_region.head()

,상권업종대분류명,시도명,시군구명,행정동명,법정동명,cnt
0,과학·기술,강원특별자치도,강릉시,강남동,노암동,20
1,과학·기술,강원특별자치도,강릉시,강남동,월호평동,2
2,과학·기술,강원특별자치도,강릉시,강남동,유산동,4
3,과학·기술,강원특별자치도,강릉시,강남동,장현동,3
4,과학·기술,강원특별자치도,강릉시,강동면,강동면,11


In [21]:
df_market_cnt = df_market_cnt_by_region.groupby(['상권업종대분류명','시도명'])['cnt'].sum().reset_index()
df_market_cnt

,상권업종대분류명,시도명,cnt
0,과학·기술,강원특별자치도,5196
1,과학·기술,경기도,61141
2,교육,강원특별자치도,3114
3,교육,경기도,28261
4,보건의료,강원특별자치도,583
5,보건의료,경기도,6318
6,부동산,강원특별자치도,2306
7,부동산,경기도,26451
8,소매,강원특별자치도,22468
9,소매,경기도,128892


In [22]:
(
    alt.
    Chart(df_market_cnt).
    encode(
        y = alt.X('상권업종대분류명', sort = '-x'),
        x = 'cnt',
        row = '시도명'
    ).
    mark_bar()
)

alt.Chart(...)

In [23]:
query = '''
    select 시군구명
	 , count(*) as cnt
	from Market_info
	where 시도명 = '경기도'
	  and 상호명 like '공차%'
	GROUP BY 시군구명
	order by 2 desc;
    '''

df_gongcha = pd.read_sql_query(query, conn)

In [24]:
## 지역별 공차
total_cnt = np.sum(df_gongcha['cnt'])

df_gongcha['ratio'] = df_gongcha['cnt'].apply(lambda x: x / total_cnt)
df_gongcha.head()


,시군구명,cnt,ratio
0,화성시,13,0.080745
1,성남시 분당구,10,0.062112
2,평택시,9,0.055901
3,파주시,9,0.055901
4,수원시 영통구,8,0.049689


In [25]:
(
    alt.
    Chart(df_gongcha).
    encode(
        x = alt.X('시군구명', sort = '-y'),
        y ='cnt',
        color = alt.condition(
            alt.FieldOneOfPredicate(field='시군구명', oneOf=['고양시 덕양구', '고양시 일산동구', '고양시 일산서구']),
            alt.value('red'),
            alt.value('lightgray')
        )
    ).
    mark_bar()
)


alt.Chart(...)

In [33]:
## 이제 여기서 뭘 찾을수 있을까 본격적으로.
# raw data check
query = '''
    select *
	from M_info
	;
    '''

df_raw_data = pd.read_sql_query(query, conn)

In [34]:
len(df_raw_data)

2338043

In [30]:
df_raw_data.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [35]:
df_raw_data['시도명'].unique()

array(['강원특별자치도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시',
       '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전북특별자치도',
       '제주특별자치도', '충청남도', '충청북도'], dtype=object)

In [ ]:
df_raw_data['']

In [36]:
df_raw_data.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA0101202210A0084547,금강산노래광장,None,I2,음식,I211,주점,I21101,일반 유흥 주점,I56211,...,4.217010e+24,파크장,강원특별자치도 동해시 송정로 11,240806,25789,None,지,None,129.127525,37.495265
1,MA010120220805430903,엔젤,None,I2,음식,I201,한식,I20101,백반/한정식,I55109,...,4.277032e+24,None,강원특별자치도 정선군 남면 민둥산로 175-13,233837,26148,None,None,None,128.792089,37.259788
2,MA010120220805430941,누베헤어,None,S2,수리·개인,S207,이용·미용,S20701,미용실,S96112,...,4.211013e+24,춘천파크자이,강원특별자치도 춘천시 스포츠타운길 460,200200,24472,None,3,None,127.707653,37.867744
3,MA010120220805430946,공차,None,I2,음식,I212,비알코올,I21201,카페,I56229,...,4.213033e+24,None,강원특별자치도 원주시 지정면 신지정로 211,220821,26354,None,None,None,127.872713,37.374189
4,MA010120220805431369,행운쉼터,None,I1,숙박,I101,일반 숙박,I10103,펜션,I55104,...,4.281032e+24,None,강원특별자치도 인제군 북면 금강로 751,252826,24609,None,None,None,128.207086,38.171871


### 자료 출처

::: {.callout-note title = "자료 출처"}
    공공 데이터 포털
    소상공인진흥공단 상가(상권)정보
    202406

:::

In [ ]:
## 밸라시타 샐러디

In [42]:
df_temp = df_raw_data[df_raw_data['시도명'] == '경기도'][['상호명','상권업종대분류명','상권업종중분류명','상권업종소분류명','층정보','도로명','경도','위도']]

In [43]:
coordinates = df_temp[['위도','경도']].to_numpy()
coordinates

array([[ 37.31916439, 127.05822893],
       [ 36.93633588, 127.04807531],
       [ 37.01803922, 127.0343809 ],
       ...,
       [ 37.69419021, 126.77024858],
       [ 37.50303422, 126.76204728],
       [ 37.71883825, 127.19738256]])

In [46]:
dbscan = DBSCAN(eps= 0.01, min_samples=5)
clusters = dbscan.fit_predict(coordinates)

In [47]:
df_temp['clu'] = clusters

In [48]:
df_temp.head()

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,층정보,도로명,경도,위도,clu
92579,엠마스키친,음식,서양식,경양식,None,경기도 용인시 수지구 성복2로,127.058229,37.319164,0
92580,팽성농산물센터,소매,식료품 소매,채소/과일 소매업,None,경기도 평택시 팽성읍 석근2길,127.048075,36.936336,0
92581,역전할머니맥주평택궁리점,음식,주점,생맥주 전문,None,경기도 평택시 고덕면 궁2길,127.034381,37.018039,0
92582,혜윰건축사사무소,과학·기술,기술 서비스,건축 설계 및 관련 서비스업,None,경기도 안산시 상록구 웃말5길,126.871046,37.306182,0
92583,양촌리민물장어,음식,한식,해산물 구이/찜,None,경기도 파주시 경의로,126.759748,37.703802,0
